# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [42]:
# Dependencies and Setup

import pandas as pd
import hvplot.pandas 
import requests
#had to install these to make a map plot
import cartopy.crs as ccrs
import geoviews as gv # noqa
import pyproj

# Import API key
from geoapify import geoapify_key
import warnings
warnings.filterwarnings("ignore")

In [43]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../weatherpy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,souillac,-20.5167,57.5167,297.36,94,40,1.03,MU,1683070771
1,1,ushuaia,-54.8000,-68.3000,280.96,76,20,6.17,AR,1683070771
2,2,anori,-3.7728,-61.6442,297.03,94,89,0.78,BR,1683070772
3,3,raudeberg,61.9875,5.1352,278.52,48,86,5.29,NO,1683070772
4,4,shimoda,34.6667,138.9500,292.35,58,92,3.28,JP,1683070772


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [44]:
# %%capture --no-display

# Configure the map plot
map_plot_1 =  city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 800,
    frame_height = 600
)
# Display the map
map_plot_1



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [45]:
# Narrow down cities that fit criteria and drop any results with null values
#not too humid
city_data_df = city_data_df.loc[city_data_df['Humidity']<40]
#not too cloudy
city_data_df = city_data_df.loc[city_data_df['Cloudiness']>50]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
print(len(city_data_df))
city_data_df.head()

16


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
66,66,aranos,-24.1333,19.1167,295.83,20,67,2.27,NaN,1683070793
71,71,fort nelson,58.8053,-122.7002,299.23,17,75,3.13,CA,1683070795
166,166,gayeri,12.6497,0.4931,305.15,28,63,2.74,BF,1683070832
172,172,dulce grande,23.0000,-102.1667,300.50,7,62,8.97,MX,1683070835
182,182,yumen,40.2833,97.2000,281.63,17,100,6.99,CN,1683070840


### Step 3: Create a new DataFrame called `hotel_df`.

In [46]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = "" 

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
66,aranos,NaN,-24.1333,19.1167,20,
71,fort nelson,CA,58.8053,-122.7002,17,
166,gayeri,BF,12.6497,0.4931,28,
172,dulce grande,MX,23.0000,-102.1667,7,
182,yumen,CN,40.2833,97.2000,17,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [50]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"Hotels",
    "apikey":geoapify_key,
    "Limit":10
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    hot_lat = row["Lat"]
    hot_lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{hot_lng},{hot_lat},{radius}"
    params["bias"] = f"proximity:{hot_lng},{hot_lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
aranos - nearest hotel: No hotel found
fort nelson - nearest hotel: No hotel found
gayeri - nearest hotel: No hotel found
dulce grande - nearest hotel: No hotel found
yumen - nearest hotel: No hotel found
manyana - nearest hotel: No hotel found
jiuquan - nearest hotel: No hotel found
kita - nearest hotel: No hotel found
hailar - nearest hotel: No hotel found
paysandu - nearest hotel: No hotel found
mandalgovi - nearest hotel: No hotel found
hay river - nearest hotel: No hotel found
linjiang - nearest hotel: No hotel found
bougouni - nearest hotel: No hotel found
otavi - nearest hotel: No hotel found
flin flon - nearest hotel: No hotel found


{'statusCode': 401, 'error': 'Unauthorized', 'message': 'Invalid apiKey'}

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [51]:
# %%capture --no-display

# Configure the map plot
map_plot_2 =  city_data_df.hvplot.points(
    "Lng",
    "Lat",
    "Hotel Name",
    "Country",
    geo = True,
    tiles = "OSM",
    color = "City",
    frame_width = 800,
    frame_height = 600
)

# Display the map
map_plot_2

TypeError: points() takes from 1 to 3 positional arguments but 5 were given